In [1]:
import pandas as pd
import numpy as np

In [2]:
#load the data
data = pd.read_csv("data/July17.csv")
data.head()

,ORIGIN,DEST,DEP_DELAY,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 9
0,ABE,DTW,-5.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ABE,DTW,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ABE,DTW,-9.0,-9.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ABE,DTW,-8.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ABE,DTW,-6.0,-19.0,NaN,NaN,NaN,NaN,NaN,NaN


var cities =
{"nodes": [{"name":"Chicago"}],
"links":[{"source":1,"target":0,"value":20}]};

In [58]:
def getCityData(city, dataset):
    city_destin_data = dataset[dataset.DEST == city]
    city_destin_data = city_destin_data.groupby('ORIGIN', as_index = False).mean()
    city_origin_data = dataset[dataset.ORIGIN == city]
    city_origin_data = city_origin_data.groupby('DEST', as_index = False).mean()
    return city_destin_data, city_origin_data

In [59]:
def dictPad(text):
    return dict({"name":text})

def getNodes(city, incoming, outgoing):
    cities = []
    for i in incoming.ORIGIN:
        cities.append(i)
    for i in outgoing.DEST:
        cities.append(i)
    cities = list(set(cities))
    return len(cities), [city]+cities+cities

def getNewNodes(city, incoming, outgoing):
    in_cities = []
    out_cities = []
    for i in incoming.ORIGIN:
        in_cities.append(i)
    for i in outgoing.DEST:
        out_cities.append(i)
    print (len(in_cities), len(out_cities))
    return len(in_cities), [city]+in_cities+out_cities

In [60]:
def getIncomingLinks(data, nodes, city):
    links = []
    for index, row in data.iterrows():
        links.append(dict({"source": nodes.index(row['ORIGIN']) ,"target":nodes.index(city) ,"value":row['ARR_DELAY'] }))
    return links

def getOutgoingLinks(data, nodes, city, num):
    links = []
    for index, row in data.iterrows():
        links.append(dict({"source": nodes.index(city),"target": max(loc for loc, val in enumerate(nodes) if val == row['DEST']) ,"value":row['DEP_DELAY'] }))
    return links


In [66]:
city = "JFK"
incoming, outgoing = getCityData(city, data)

incoming = incoming.sort_values('ARR_DELAY', axis=0, ascending=False)
outgoing = outgoing.sort_values('DEP_DELAY', axis=0, ascending=False)

num, nodes = getNewNodes(city, incoming, outgoing)

links_in = getIncomingLinks(incoming, nodes, city)
links_out = getOutgoingLinks(outgoing, nodes, city, num)
links = links_in + links_out


57 56


In [67]:
cities = {"nodes": list(map(lambda x:dictPad(x),nodes)), "links": links}

In [68]:
import json
file = 'data_{0}.txt'.format(city)
with open(file, 'w') as outfile:
    json.dump(cities, outfile)

In [69]:
# newlist = sorted(sorted(links, key=lambda m: m['target']) , key=lambda k: k['value']) 
# newlist

In [70]:
# cities = {"nodes": list(map(lambda x:dictPad(x),nodes)), "links": newlist}
# import json
# file = 'data_{0}.txt'.format(city+"sort")
# with open(file, 'w') as outfile:
#     json.dump(cities, outfile)

In [71]:
def visualize(cities, nodes):
    for idx,l in enumerate(cities['links']):
        print (1+idx, nodes[l['source']] + "-->" + nodes[l['target']] + "=" + str(l['value']))
        
visualize(cities,nodes)

1 SJC-->JFK=73.48387096774194
2 SMF-->JFK=58.241379310344826
3 DCA-->JFK=50.516129032258064
4 PHL-->JFK=50.193548387096776
5 RSW-->JFK=44.18032786885246
6 FLL-->JFK=39.91694352159468
7 ORD-->JFK=38.89041095890411
8 RDU-->JFK=38.63793103448276
9 HOU-->JFK=37.357142857142854
10 MCO-->JFK=32.174107142857146
11 PSE-->JFK=30.0
12 MIA-->JFK=29.924342105263158
13 CLT-->JFK=29.91891891891892
14 PBI-->JFK=28.5
15 ACK-->JFK=27.77777777777778
16 BUF-->JFK=27.663366336633665
17 JAX-->JFK=26.65909090909091
18 MVY-->JFK=25.63157894736842
19 SAV-->JFK=25.17543859649123
20 BGR-->JFK=22.727272727272727
21 LGB-->JFK=22.666666666666668
22 ATL-->JFK=21.826291079812208
23 ROC-->JFK=21.261904761904763
24 BUR-->JFK=20.774193548387096
25 ABQ-->JFK=19.612903225806452
26 PWM-->JFK=19.36521739130435
27 BOS-->JFK=19.26843657817109
28 SJU-->JFK=19.13372093023256
29 TPA-->JFK=19.085106382978722
30 LAS-->JFK=17.885416666666668
31 IAD-->JFK=17.8625
32 AUS-->JFK=17.30635838150289
33 SFO-->JFK=16.989583333333332
34 MSY